In [52]:
import os
import cv2
import imutils
import numpy as np

In [53]:
def showImage(img, img_alt="img"):
  cv2.imshow(img_alt, img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [177]:
count = -1
cl = [
  (255,255,0), (0,255,255), (255,0,255), (255,255,255), (255,127,80), (100,149,237)
]
def get_cl(count):
  return cl[count%len(cl)]

In [27]:
class ShapeDetector:
  
  def __init__(self):
    self.count = -1
  
  def detect(self, c):
    
    shape = "undefined"
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.04 * peri, True)
    shapeLen = len(approx)
    if shapeLen == 3:
      shape = "triangle"
    elif shapeLen == 4:
      shape = "square"
      (x, y, w, h) = cv2.boundingRect(approx)
      ar = w / float(h)
      shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
    elif shapeLen == 5:
      shape = "pentagon"
    elif shapeLen == 6:
      shape = "hexagon"
    elif shapeLen == 7:
      shape = "heptagon"
    elif shapeLen == 8:
      shape = "octagon"
    else:
      shape = "circle"
      
    return shape
  
  def preProcess(self, img):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    showImage(gray, "gray")
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    showImage(blurred, "blurred")
    # thresh = cv2.threshold(blurred, 100, 200, cv2.THRESH_OTSU)[1]
    edges = cv2.Canny(blurred, 20, 150)
    showImage(edges, "edges")
    dilated = cv2.dilate(edges.copy(), np.ones((7, 7), np.uint8), iterations=1)
    showImage(dilated, "dilated")
    eroded = cv2.erode(dilated, np.ones((3, 3), np.uint8), iterations=3)
    # eroded = cv2.erode(edges.copy(), np.ones((3, 3), np.uint8), iterations=1)
    showImage(eroded, "eroded")
    cnts = cv2.findContours(eroded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    minArea = 2000
    img_copy = img.copy()
    obj_rects = []
    for c in cnts:
      # shape = self.detect(c)
      area = cv2.contourArea(c)
      print(f"TYPE : {type(c)}, SHAPE: {c[0]}")
      
      if area > minArea:
        obj_rects.append(self.getRect(c.copy()))
        
        epsilon = 0.06 * cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, epsilon, True)
        cv2.drawContours(img_copy, [c], -1, get_cl(self.count), 3)
        self.count += 1
        # if len(approx) > 4:
        #   approx = cv2.approxPolyDP(c, cv2.arcLength(c, True) * 0.02, True)
        #   approx = cv2.approxPolyDP(approx, epsilon, True)
        # if len(approx) == 4:
        #   cv2.drawContours(img, [approx], -1, (0, 255, 0), 2)
        # else:
        #   cv2.drawContours(img, [approx], -1, get_cl(self.count), 1)
        #   self.count+=1
        
        
      else:
        cv2.drawContours(img_copy, [c], -1, (255, 0, 0), 1)
    
    showImage(img_copy, "final")
    self.draw_rects(img, obj_rects)
  
  def getRect(self, points):
    reshaped_points = points.reshape(-1, 2)
    min_x = np.min(reshaped_points[:, 0])
    min_y = np.min(reshaped_points[:, 1])
    max_x = np.max(reshaped_points[:, 0])
    max_y = np.max(reshaped_points[:, 1])
    
    return [[min_x, min_y], [max_x, max_y]]
  
  def draw_rects(self, img, rectPoints):
    for rect in rectPoints:
      cv2.rectangle(img, rect[0], rect[1], (0, 255, 0), 3)
      showImage(cv2.cvtColor(img, cv2.COLOR_RGB2BGR), "rects")

In [366]:
PROJECT_DIR = "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis"
IMG_MAIN_PATH = os.path.join(PROJECT_DIR, "src/main/resources/sampleImages/set1")
imageNames = ["pan-gen.jpg"]

In [367]:
obj = ShapeDetector()
for imageName in imageNames:
  imgPath = os.path.join(IMG_MAIN_PATH, imageName)
  img = cv2.imread(imgPath)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  showImage(img, "initial")
  img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_AREA)
  obj.preProcess(img)

TYPE : <class 'numpy.ndarray'>, SHAPE: [[428 369]]
TYPE : <class 'numpy.ndarray'>, SHAPE: [[291 348]]
TYPE : <class 'numpy.ndarray'>, SHAPE: [[357 116]]
TYPE : <class 'numpy.ndarray'>, SHAPE: [[383   0]]
TYPE : <class 'numpy.ndarray'>, SHAPE: [[313   0]]
TYPE : <class 'numpy.ndarray'>, SHAPE: [[0 0]]


In [17]:
import yaml
import os

In [18]:
yaml_path=os.path.abspath("/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis/src/main/resources/properties.yaml")

In [29]:
with open(yaml_path, 'r') as rf:
  data = yaml.safe_load(rf)

In [30]:
data

{'DetectRacks': {'blur': {'kernal': [7, 7]},
  'edge': {'canny': {'tLower': 20,
    'tUpper': 150,
    'apertureSize': 3,
    'l2Gradient': True}},
  'dilate': {'Kernel': [7, 7], 'iterations': 1},
  'erode': {'Kernel': [3, 3], 'iterations': 3},
  'minArea': 2000,
  'boundingBox': {'color': [0, 255, 0], 'thickness': 1}}}

In [32]:
type(data['DetectRacks']['edge']['canny']['l2Gradient'])

bool

In [28]:
sb = ShapeDetector()
sb.__class__.__name__

'ShapeDetector'

In [64]:
import os
import sys

In [65]:
module_paths = [
  "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis/src/main/python",
               ]

In [67]:
for module_path in module_paths:
  if module_path not in sys.path:
    sys.path.append(module_path)

In [68]:
import utils.loadProperties as lp
propertyFilePath = "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis/src/main/resources/properties.yaml"

In [69]:
data = lp.loadProperties(propertyFilePath)

In [70]:
data.keys()

dict_keys(['DetectRacks'])

In [71]:
data = lp.loadProperties(propertyFilePath, True, "DetectRacks")

In [72]:
data

{'blur': {'kernal': [7, 7]},
 'edge': {'canny': {'tLower': 20,
   'tUpper': 150,
   'apertureSize': 3,
   'l2Gradient': True}},
 'dilate': {'Kernel': [7, 7], 'iterations': 1},
 'erode': {'Kernel': [3, 3], 'iterations': 3},
 'minArea': 2000,
 'boundingBox': {'color': [0, 255, 0], 'thickness': 1}}

In [1]:
import os
import sys

os.environ["PROPERTY_FILE_PATH"] = "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis/src/main/resources/properties.yaml"

os.environ["ENVIRONMENT"] =  "local"

module_paths = [
  "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis/src/main/python",
               ]

for module_path in module_paths:
  assert os.path.exists(module_path), f"module path {module_path} does not exist"
  
  if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
PROJECT_DIR = "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis"
IMG_MAIN_PATH = os.path.join(PROJECT_DIR, "src/main/resources/sampleImages/set1")
imageNames = ["p1.jpg"]


In [3]:
# rack detector driver
import cv2
import utils.loadProperties as lp
import utils.localRunHelpers as lr
from RacksDetection import DetectRacks


commonProps=lp.loadProperties(os.getenv("PROPERTY_FILE_PATH"),True,"Common")

detector = DetectRacks.DetectRacks()
for imageName in imageNames:
  imgPath = os.path.join(IMG_MAIN_PATH, imageName)
  img = cv2.imread(imgPath)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  lr.showImage(img, "initial")
  img = cv2.resize(img, commonProps["resizeKernal"], interpolation=cv2.INTER_AREA)
  lr.showImage(img, "resized")
  rectangles = detector.findBoxes(img)
  print(rectangles)

[[[367, 195], [484, 436]], [[14, 153], [505, 511]], [[14, 0], [503, 163]]]


In [13]:
type(rectangles)

NoneType

{'blur': {'kernal': [7, 7], 'sigmaX': 0},
 'edge': {'canny': {'tLower': 20,
   'tUpper': 150,
   'apertureSize': 3,
   'l2Gradient': False}},
 'dilate': {'Kernel': [7, 7], 'iterations': 1},
 'erode': {'Kernel': [3, 3], 'iterations': 3},
 'minArea': 2000,
 'boundingBox': {'color': [0, 255, 0], 'thickness': 1}}

In [6]:
import os
import base64
PROJECT_DIR = "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis"
IMG_MAIN_PATH = os.path.join(PROJECT_DIR, "src/main/resources/sampleImages/set1")
imageNames = ["p1.jpg"]

In [12]:
image_path = os.path.join(IMG_MAIN_PATH, imageNames[0])
with open(image_path, 'rb') as rf:
  data = rf.read()

b64data = base64.b64encode(data).decode('utf-8')
with open("b64_img.txt", "w") as wf:
  wf.write(b64data)

In [6]:
import os
import base64
import cv2
import numpy as np
import json
PROJECT_DIR = "/Users/vinodpatil/LAB/UPGRAD/Masters/FINAL_THESIS_WRITING/APP/remi-wireframe-synthesis"
IMAGE_PATH = "src/main/Rough/recieved.txt"
b64_path = os.path.join(PROJECT_DIR, IMAGE_PATH)
def showImage(img, img_alt="img"):
  cv2.imshow(img_alt, img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [4]:
with open(b64_path, 'r') as rf:
  bytes = rf.read()
  data = base64.b64decode(bytes)
  nd_array = np.frombuffer(data, dtype=np.uint8)
  img = cv2.imdecode(nd_array, cv2.IMREAD_COLOR)
  showImage(img)
  

In [7]:
RESPONSE_FILE = os.path.join(PROJECT_DIR, "src/main/Rough/response.json")
with open(RESPONSE_FILE, 'r') as rf:
  json_data = json.load(rf)
  points = json_data["coordinates"]
  bytes = json_data["image"]
  data = base64.b64decode(bytes)
  nd_array = np.frombuffer(data, dtype=np.uint8)
  img = cv2.imdecode(nd_array, cv2.IMREAD_COLOR)
  for point in points:
    cv2.rectangle(img, point[0], point[1], (0, 255, 0), 2)
  showImage(img)

In [2]:
def print_args(*args, **kwargs):
  if args:
    print(args[0])

print_args("qwqwq", 2, 3, "hello")

qwqwq
